In [1]:
# importing necessary dependencies
import pandas as pd
from datetime import datetime, timedelta

In [2]:
zulo_bank = pd.read_csv('Dataset/zulo_bank.csv')

In [5]:
zulo_bank.head(5)

,FullName,Email,Phone,TransactionType,Amount,TransactionDate,AccountType,Balance,OpeningDate,LoanAmount,LoanType,StartDate,EndDate,InterestRate
0,Carol Miller,yfisher@example.org,6088279027,withdrawal,102.15,2023-04-26,Savings,5652.16,2019-08-12,NaN,NaN,NaN,NaN,NaN
1,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,32428.90,Mortgage,2021-06-24,2050-01-08 04:59:17.907588,2.12
2,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,31406.77,Personal,2021-02-27,2038-10-12 04:59:17.907821,4.63
3,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,27834.00,Personal,2019-12-05,2037-08-15 04:59:17.909497,2.15
4,Geoffrey Banks,gonzalesgeorge@example.net,001-546-857-6518x5359,withdrawal,358.80,2020-06-13,Credit,2881.24,2019-05-06,27873.08,Auto,2022-01-19,2037-06-03 04:59:17.913974,7.03
